In [2]:
from config import preprocessing as conf_preprocess
import utils

In [3]:
from scipy import signal
from tqdm import tqdm
import os
import argparse
import time
import csv
import soundfile as sf
import numpy as np
import librosa
import numpy as np
import tensorflow as tf
import time
import librosa.display as display
import numpy as np
from python_speech_features import mfcc
from six.moves import range
import scipy.io.wavfile as wav
import sys

In [22]:
def mfcc(audio , sr , hop_length , win_length):
    axis = -1
    mean = np.mean(audio, axis=axis)
    std_dev = np.std(audio, axis=axis) + 1e-9
    normalized = (audio - mean) / std_dev
    mfcc = librosa.feature.mfcc(normalized,sr =sr , n_mfcc=80)
    mfcc = librosa.amplitude_to_db(mfcc)
    return mfcc.T

In [5]:
signal, sr = librosa.load('/home/u52660/data/25hours/wavs/0/0/00000.wav')

In [6]:
signal.shape

(114688,)

In [7]:
audio = signal 
axis = -1
mean = np.mean(audio, axis=axis)
std_dev = np.std(audio, axis=axis) + 1e-9
normalized = (audio - mean) / std_dev

In [12]:
k=librosa.feature.mfcc(normalized , n_mfcc = 40)

In [4]:
k

NameError: name 'k' is not defined

In [14]:
k = librosa.amplitude_to_db(k)

In [16]:
k.shape

(40, 225)

In [9]:
normalized

array([-0.00010834, -0.00010834, -0.00010834, ...,  0.00439955,
        0.0029367 ,  0.00026383], dtype=float32)

In [10]:
audio

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.3999905e-04,
       3.6476480e-04, 4.4582503e-05], dtype=float32)

In [254]:
normalized.shape

(114688,)

In [8]:
f = os.walk('/home/u52660/data/25hours/')

In [6]:
num_hours = 0
character_mapping = utils.create_character_mapping()

In [9]:
start_time = time.time()
with open(os.path.join(conf_preprocess['data_dir'], 'metadata.csv'), 'w', newline='') as metadata:
    metadata_writer = csv.DictWriter(metadata, fieldnames=['filename', 'spec_length', 'labels_length', 'labels'])
    metadata_writer.writeheader()
    for root , dirs , files in tqdm(f):
        for file in files :
            if file[-4:] == '.txt':
                  with open(os.path.join(root, file), 'r') as f:
                        for line in f.readlines():
                            sections = line.split('|')
                            serious = sections[0].split('/')
                            kali = serious
                            path_save = '/home/u52660/data/25hours/spec/'+str(sections[0]) + '.npy'
                            audio, sr = librosa.load(os.path.join(root + 'wavs/' , sections[0]))
                            audio = librosa.resample(audio , sr , 16000)
                            num_hours += (len(audio) / sr) / 3600 
                            spec = mfcc(audio , 16000 , hop_length=400 , win_length=160 )
                            #print(spec.shape)
                            np.save(path_save, spec ,allow_pickle=False)
                            ids = [character_mapping[c] for c in '|'.join(sections[1:]).lower()
                                       if c in character_mapping]
                            k = [str(i) for i in ids]
                            metadata_writer.writerow({
                                'filename': sections[0],
                                'spec_length': spec.shape[0],
                                'labels_length': len(ids),
                                'labels': ' '.join([str(i) for i in ids])
                            })                            
print('DONE')
print("Hours pre-processed: " + str(num_hours))
print("Time: " + str(time.time() - start_time))

['scripts.csv', 'scripts.txt', 'metadata.csv']
DONE
Hours pre-processed: 0
Time: 0.006566524505615234
